Let's try to predict how much money we should budget for sales commissions for the next year.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

Assumptions:
- historical percent to target performance is centered around a mean of 100%
- std dev 10%
- 500 sales reps
- 1000 simulations

In [2]:
avg = 1
std_dev = .1
num_reps = 500
num_simulations = 1000

In [3]:
pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)
print(pct_to_target[0:10])

[1.04 1.11 0.86 0.98 0.89 0.85 1.03 0.82 0.81 1.03]


In [4]:
sales_target_values = [75_000, 100_000, 200_000, 300_000, 400_000, 500_000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)

In [5]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                               'Sales_Target': sales_target})

df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']
df.head()

,Pct_To_Target,Sales_Target,Sales
0,1.04,75000,78000.0
1,1.11,200000,222000.0
2,0.86,75000,64500.0
3,0.98,75000,73500.0
4,0.89,200000,178000.0


In [6]:
# Mapping Pct_To_Target to commission rate
def calc_commission_rate(x):
    """ Return the commission rate based on the table:
    0-90% = 2%
    91-99% = 3%
    >= 100 = 4%
    """
    if x <= .90:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

In [7]:
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']
df.head()

,Pct_To_Target,Sales_Target,Sales,Commission_Rate,Commission_Amount
0,1.04,75000,78000.0,0.04,3120.0
1,1.11,200000,222000.0,0.04,8880.0
2,0.86,75000,64500.0,0.02,1290.0
3,0.98,75000,73500.0,0.03,2205.0
4,0.89,200000,178000.0,0.02,3560.0


In [8]:
# Running simulation

# Define a list to keep all the results from each simulation that we want to analyze
all_stats = []

# Loop through many simulations
for i in range(num_simulations):

    # Choose random inputs for the sales targets and percent to target
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

    # Build the dataframe based on the inputs and number of reps
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

    # Back into the sales number using the percent to target rate
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

    # Determine the commissions rate and calculate it
    df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

    # We want to track sales,commission amounts and sales targets over all the simulations
    all_stats.append([df['Sales'].sum().round(0),
                      df['Commission_Amount'].sum().round(0),
                      df['Sales_Target'].sum().round(0)])

In [9]:
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                           'Commission_Amount',
                                                           'Sales_Target'])

In [10]:
results_df.describe().style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"1,000.0","1,000.0","1,000.0"
mean,"83,699,782.5","2,858,402.84","83,678,550.0"
std,"2,714,754.492979227","102,316.8642696128","2,684,007.972472566"
min,"74,583,750.0","2,561,860.0","74,300,000.0"
25%,"81,924,062.5","2,789,415.75","81,893,750.0"
50%,"83,699,250.0","2,857,796.0","83,687,500.0"
75%,"85,541,562.5","2,925,264.75","85,450,000.0"
max,"94,343,000.0","3,242,155.0","93,850,000.0"


We can see that the average commission expense is 2.85M and the standard deviation is 104K.
The commission expense can be as low as 2.5M, or as high as 3.1M.